In [ ]:
import pandas as pd
import numpy as np
import requests

import json

pd.options.display.max_rows = 400
pd.options.display.max_colwidth = 4000

In [ ]:
df_searches = pd.read_excel("searches_candidates_selection_rank.xlsx")
df_searches = df_searches[df_searches['query'] != 'undefined']

In [ ]:
def slug_only(dr):
    candidates = json.loads(dr.replace("'", '"'))    
    try :
        return json.dumps([c['slug'] for c in candidates])
    except TypeError:
        print('ERROR:\n' + dr)
        return None

df_searches['docs_slug'] = df_searches.docs_res.apply(lambda dr : slug_only(dr))

In [ ]:
df_searches_unique = df_searches[["rank", "docs_slug", "query", "docs_res", "idVisit"]]
df_searches_unique.drop_duplicates(["docs_slug", "query", "idVisit"], inplace=True)

In [ ]:
df_searches.shape

# Evaluation : group queries and get rank for each candidates 
Objective : compute NDCG / MAP / MRR

In [ ]:
# most frequent queries with hit
all_ranks = []

threshold = 5
for g, rows in df_searches_unique[df_searches['rank'] >= 0].groupby(["docs_slug"]):
    try:
        n = rows.shape[0]
        q = set(rows['query'].tolist())    
        algos = [docres['algo'] for docres in json.loads(rows.docs_res.tolist()[0].replace("'",'"'))]
        #algos = 
        preq = 'pre-qualified' in algos
        candidates = g
        if (n > threshold):        
            print(f'{q} -> {n}')
            #print(algo)
            ranks = rows['rank'].value_counts().sort_index()
            candidates_array = json.loads(candidates)
            j = len(candidates_array)
            complete_ranks = [0] * j
            ids = []
            for i in range(0, j):
                if i in ranks.index:
                    complete_ranks[i] = ranks[i]             
                ids.append({'slug':candidates_array[i], 'count':int(complete_ranks[i]), 'preq':algos[i]})
            all_ranks.append({'total':int(n), 'queries':list(q), 'preq': preq, 'docs':ids})
    except KeyError:
        print("Key error : " + g)
        continue

In [ ]:
print(len(all_ranks))
#print()\
#print(sorted(all_ranks[6], key=lambda x: x['count'], reverse=True))

In [ ]:
print(json.dumps(all_ranks[15], indent=2))

In [ ]:
# from documentation

params = {
    "y_pred" : ["a", "b", "c", "w", "k","e"], # y_pred (array): list of documents id predicted by the system
    "y_true" : ["a", "b", "c", "e"], # y_true (array): documents id scored by humans sorted from most relevant to least relevant
    "y_score" : {
        "a":5,
        "b":3,
        "c":2,
        "e":2
            }, # y_score is a dict {"doc_id":"score"} of documents assigned as relevant y humans with the associated scores (ordered internally)
    "method" : "all", # one of ["precision", "recall", "dcg", "mrr", "all"]
    "k": 3 # k integer preferaly =< to length(y_pred)
}

In [ ]:
def sort_ranks(ranks):
    return sorted(ranks, key=lambda x: x['count'], reverse=True)


In [ ]:
# not used anymore
'''
def compute_metrics(rank_results, i, verbose=False, api=False):
    scores = dict([(r['slug'], int(r['count'])) for r in rank_results[i]['docs']])
    if not api:
        y_pred = [c['slug'] for c in rank_results[i]['docs']]
    elif api:
        y_pred = rank_results[i]['docs_api']
        
    params = {

        "y_pred": y_pred,
        #"y_pred": [c['slug'] for c in sort_ranks(all_ranks[i]) if c['count'] > 0],
        "y_true" : [c['slug'] for c in sort_ranks(rank_results[i]['docs']) if c['count'] > 0],
        "y_score" : scores,
        "method" : "all",
        "k":5
    }
    r = requests.post("http://localhost:4545/api/score", json = params)
    
    if r.status_code != 200:
        print("Error with at index " + str(i))        
        result = {'dcg': 1.0, 'mrr': 1.0, 'precision': 1.0, 'recall': 1.0}
    else :
        result = r.json()
        
    if verbose:
        print(i)
        print(rank_results[i][0]['queries'])
        print(json.dumps(scores, indent=2))
        print(result)
    
    result['occ'] = rank_results[i]['total']
    return result
'''

In [ ]:
# Borrowed from Armand's letor
def discounted_cumulative_gain(y_score, y_true, y_pred, k):
    """y_score is a dictionnary {"doc_id":"score"} of documents assigned as relevant y humans with the associated scores
       k = maximum index to be scored
       /!\ to be valid, dcg should have results lists of same length bbetween requests"""
    if y_pred == []:
        return 0
    if k > len(y_pred):
        k = len(y_pred)
    
    y_scoring = []
    for y in y_pred:
        score = y_score.get(y)
        if score is None: # if the predicted document is not in the array of humanly scored documents
            score = 0
        y_scoring.append(score)
    
    # compute dcg
    def compute_dcg(y_scoring, k):
        dcg = []
        for i, pred in enumerate(y_scoring[:k]):
            i+=1
            # two formulas possible, according to wikipedia the latter places stronger emphasis on retrieving relevant documents
            dcg.append(pred/(np.log2(i)+1))
            #dcg.append((np.power(pred,2) - 1)/(np.log2(i)+1))
        return np.sum(dcg)
    # compute Ideal DCG
    # ideal DCG: the best score that could have been obtained given the relevant document list
    # i. e. : the most relevant documents, ordered by relevance.

    dcg = compute_dcg(y_scoring, k)

    ideal_scores = []
    for i in y_true:
        score = y_score.get(i)
        if not i in y_score : # if we don't have a score for y true
            y_score[i]
            raise ValueError("the true document does not have score in y_score : " + i)
        ideal_scores.append(score)

    ideal_scores = sorted(ideal_scores, reverse=True)

    ideal_dcg = compute_dcg(ideal_scores, k)
    return dcg/ideal_dcg

# Update candidates with API

In [ ]:
# for each group, trigger query 
all_ranks[113]

In [ ]:
all_scores.iloc[113]

In [ ]:
#[ar for ar in range(len(all_ranks)) if 'préavis cdd' in all_ranks[ar]['queries']]

In [ ]:
def reformat_slug(source, slug) :
    formated_source = source.replace('_', '-', )
    #print(formated_source)
    #print(source)
    formated_source = formated_source.replace('fiches', 'fiche')
    if (source == 'contributions'):
        formated_source = formated_source[:-1]
        
    return f"/{formated_source}/{slug}"

def call_api(query):
    r = requests.get("http://localhost:1337/api/v1/search?q=" + query)
    if r.status_code != 200:
        print("Error when calling search API for query : " + str(q))        
        return {}
    else :
        result = r.json()
        return result

In [ ]:
def control_call(i):
    try:
        api_res = call_api(all_ranks[i]['queries'][0])['documents']       
        call_res = [reformat_slug(d['source'], d['slug']) for d in api_res]
        origin_res = [d['slug'] for d in all_ranks[i]['docs']]
        #if (call_res != origin_res):
        #    return False
            #print([call_res[j] == origin_res[j] for j in range(len(call_res))])
            #print(json.dumps(call_res, indent=2))
            #print(json.dumps(origin_res, indent=2))

        #else:
        all_ranks[i]['docs_api'] = call_res
        return True
    except KeyError:
        return False
    except IndexError:
        return False


In [ ]:
j = 0
issues = []
for i in range(0, len(all_ranks)):
#for i in range(0, 2):
    if not control_call(i):
        issues.append(i)
len(issues)

In [ ]:
non_preq_issues = []
for i in issues:
    if not all_ranks[i]['preq']:
        #print(i)
        non_preq_issues.append(i)
len(non_preq_issues)

In [ ]:
# for now we'll only consider calls without issues

In [ ]:
no_issues_ranks = [j for i, j in enumerate(all_ranks) if i not in issues]


In [ ]:
len(all_ranks)

In [ ]:
len(no_issues_ranks)

In [ ]:
no_issues_ranks[:2]

In [ ]:
[i for i in range(0, len(no_issues_ranks)) if not 'docs_api' in no_issues_ranks[i].keys()]

In [ ]:
len(issues)

In [ ]:
def compute_dcg_local(rank_results, i, verbose=False, api=False):
    scores = dict([(r['slug'], int(r['count'])) for r in rank_results[i]['docs']])
    is_preq = rank_results[i]['preq']
    occ = sum([c for (_,c) in scores.items()])
    if not api:
        y_pred = [c['slug'] for c in rank_results[i]['docs']]
    elif api:
        y_pred = rank_results[i]['docs_api']
    y_true = [c['slug'] for c in sort_ranks(rank_results[i]['docs'])]
    ndcg = discounted_cumulative_gain(scores, y_true, y_pred, 7)
    if verbose :
        print(json.dumps(scores, indent=2))
        print(json.dumps(y_pred, indent=2))
        print(json.dumps(y_true, indent=2))
        print(ndcg)
    return {'dcg': ndcg, 'occ': occ, 'preq': is_preq}

In [ ]:
# retest all

j = 0
issues = []
#for i in range(0, len(all_ranks)):
#    if not control_call(i):
#        issues.append(i)
len(issues)
all_scores = pd.DataFrame([compute_dcg_local(no_issues_ranks, i, api=False) for i in range(len(no_issues_ranks))])
all_scores['weighted_dcg'] = all_scores['dcg'] * all_scores['occ'] 

preq_scores = all_scores[all_scores.preq]
other_scores = all_scores[~all_scores.preq]

print(f"nDCG all : {compute_weighted_dcg(all_scores)}")
print(f"nDCG preq : {compute_weighted_dcg(preq_scores)}")
print(f"nDCG other : {compute_weighted_dcg(other_scores)}")

In [ ]:
# TODO : we need to ponderate DCGs on count (as DCG of a frequent query should be higher than other)

In [ ]:
all_scores_api = pd.DataFrame([compute_dcg_local(no_issues_ranks, i, api=True) for i in range(len(no_issues_ranks))])
all_scores_api['weighted_dcg'] = all_scores_api['dcg'] * all_scores_api['occ'] 
all_scores["weighted_api"] = all_scores_api["weighted_dcg"]
all_scores["dcg_api"] = all_scores_api["dcg"]

In [ ]:
all_scores[~all_scores.preq].sort_values(by=['occ', 'dcg'], ascending=False)[:50]

In [ ]:
all_scores[~all_scores.preq]['occ'].sum()

In [ ]:
all_scores[all_scores.preq]['occ'].sum()

In [ ]:
id = 269
no_issues_ranks[id]

In [ ]:
all_scores.loc[id]

In [ ]:
compute_dcg_local(no_issues_ranks, id, verbose=True, api=True)

In [ ]:

def compute_weighted_dcg(scores):
    return round(scores['weighted_dcg'].sum() / scores['occ'].sum(), 2)


In [ ]:
preq_scores = all_scores[all_scores.preq]
other_scores = all_scores[~all_scores.preq]

print(f"nDCG all : {compute_weighted_dcg(all_scores)}")
print(f"nDCG preq : {compute_weighted_dcg(preq_scores)}")
print(f"nDCG other : {compute_weighted_dcg(other_scores)}")

In [ ]:
preq_scores = all_scores[all_scores.preq]
other_scores = all_scores[~all_scores.preq]

print(f"nDCG all : {compute_weighted_dcg(all_scores)}")
print(f"nDCG preq : {compute_weighted_dcg(preq_scores)}")
print(f"nDCG other : {compute_weighted_dcg(other_scores)}")

In [ ]:
# index of ranks with pre-qualified
# index of ranks without

preq_indices = [i for i in range(0, len(no_issues_ranks)) if no_issues_ranks[i]['preq']]
other_indices = [i for i in range(0, len(no_issues_ranks)) if not no_issues_ranks[i]['preq']]

In [ ]:
print(len(preq_indices))
print(len(other_indices))

In [ ]:
#preq_scores = [compute_dcg_local(no_issues_ranks, i) for i in preq_indices]
#other_scores = [compute_dcg_local(no_issues_ranks, i, api=False) for i in other_indices]

In [ ]:
#all_scores = [compute_metrics(no_issues_ranks, i, api=True) for i in range(len(no_issues_ranks))]

In [ ]:
# compute weighted average
# show : all / preq only / other / fulltex / sem

In [ ]:
# show : only fulltext / only sem

In [ ]:
#pd.DataFrame(all_scores)['dcg'].describe()

In [ ]:
#pd.DataFrame(all_scores)['dcg'].describe()

In [ ]:
#pd.DataFrame(preq_scores)['dcg'].describe()

In [ ]:
#pd.DataFrame(other_scores)['dcg'].describe()

In [ ]:
#pd.DataFrame(preq_scores + other_scores)['dcg'].describe()

In [ ]:
run_preq = False
if run_preq:
    scores = preq_scores
    indices = preq_indices
else:
    scores = other_scores
    indices = other_indices

min_score = 0.98
min_occ = 5

to_check_indices = [(i, round(scores.iloc[i]['dcg'], 2), scores.iloc[i]['occ']) for i in range(0, len(scores)) if scores.iloc[i]['dcg'] < min_score and scores.iloc[i]['occ'] > min_occ]


In [ ]:
# sanity check
test_i = to_check_indices[10][0]
pi = indices[test_i]
print(scores.iloc[test_i])
print(json.dumps(no_issues_ranks[pi], indent=2))

In [ ]:
def format_entry(i, queries, candidates_rank, score, count):
    lines = []
    lines.append(f"## ({i}) {list(queries)[0]}")
    lines.append(f'Score : `{score}`')
    lines.append(f'Count : `{count}`\n')
    lines.append('Variantes :')
    lines.extend(['- ' + q for q in queries])
    lines.append('')
    lines.append('|count|slug|algo|')
    lines.append('|--|--|--|')
    lines.extend([f"{ar['count']} | {ar['slug']} | {ar['preq']}" for ar in candidates_rank['docs']])
    lines.append('\n')
    #lines.extend([f"{ar}" for ar in candidates_rank['docs_api']])
    lines.append('\n\n')
    return lines



all_lines = []

# sort by count
to_check_indices.sort(key=lambda x: x[2], reverse=True)

for j in range(len(to_check_indices)) :
    (i, score, count) = to_check_indices[j]
    candidates_rank = no_issues_ranks[indices[i]]
    queries = candidates_rank['queries']
    
    all_lines.extend(format_entry(j, queries, candidates_rank, score, count))    

#print("\n".join(all_lines))
#print(all_lines)
#[print(f"{ar['count']} => {ar['slug']}") for ar in candidates_rank]

In [ ]:
with open("df-updates2.md", 'w+') as f :
    f.write("\n".join(all_lines))

In [ ]:
with open("search-issues-api2.md", 'w+') as f :
    f.write("\n".join(all_lines))

In [ ]:
all_ranks[51]

# Check best semantic entries

In [ ]:
def sem_best(i):
    docs = no_issues_ranks[i]['docs']
    docs.sort(key=lambda x: x['count'], reverse=True)
    if (len(docs) > 1):
        return (docs[0]['preq'] == 'semantic') # or (docs[1]['preq'] == 'semantic')
    else :
        return False

sem_matches = [i for i in range(0, len(no_issues_ranks)) if sem_best(i)]

In [ ]:
len(sem_matches)

In [ ]:
[(no_issues_ranks[i]['queries'][0], no_issues_ranks[i]['docs'][0]['count'], call_api(no_issues_ranks[i]['queries'][0])['documents'][0]['_score']) for i in sem_matches]

In [ ]:
no_issues_ranks[211]